In [1]:
import pandas as pd

In [2]:
path = "/mnt/d/Hanze_Groningen/STAGE/data/EGAD00001000292/EGAD00001000292/delimited_maps/Sample_File.map"
df = pd.read_csv(path, sep='\t', header=None)
df.head()

,0,1,2,3
0,S1_FL,EGAN00001088435,SS6004099.bam.cip,EGAF00000151605
1,S1_GL,EGAN00001088434,SS6004094.bam.cip,EGAF00000151604
2,S1_tFL,EGAN00001088436,SS6004104.bam.cip,EGAF00000151606
3,S2_FL1,EGAN00001088438,SS6004113.bam.cip,EGAF00000151608
4,S2_FL2,EGAN00001088439,SS6004114.bam.cip,EGAF00000151609


In [3]:
df_selection = pd.DataFrame()
df_selection[['Sample','type']] = df[0].str.split("_",expand=True,)
df_selection['Name'] = df[2].str.split(".",expand=True,)[0]
df_selection['type2'] = df_selection['type'].replace(['tFL','FL', 'FL1', 'FL2', 'FL3'],'tumor').replace(['GL'],'normal')
df_selection.head()

,Sample,type,Name,type2
0,S1,FL,SS6004099,tumor
1,S1,GL,SS6004094,normal
2,S1,tFL,SS6004104,tumor
3,S2,FL1,SS6004113,tumor
4,S2,FL2,SS6004114,tumor


In [4]:
dict_sample = dict()

for index, row in df_selection.iterrows():
    if row['Sample'] in dict_sample.keys():
        if row['type2'] == 'normal':
            dict_sample[row['Sample']]['normal'].append(row['Name'])
        elif row['type2'] == 'tumor':
            dict_sample[row['Sample']]['tumor'].append(row['Name'])
    else:
        tumor = list()
        normal = list()
        if row['type2'] == 'normal':
            normal.append(row['Name'])
        elif row['type2'] == 'tumor':
            tumor.append(row['Name'])
        dict_sample[row['Sample']] = {'normal': normal, 'tumor': tumor}
dict_sample

{'S1': {'normal': ['SS6004094'], 'tumor': ['SS6004099', 'SS6004104']},
 'S2': {'normal': ['SS6004109'],
  'tumor': ['SS6004113', 'SS6004114', 'SS6004118', 'SS6004119']},
 'S3': {'normal': ['SS6004123'], 'tumor': ['SS6004124', 'SS6004129']},
 'S4': {'normal': ['SS6004128'], 'tumor': ['SS6004133', 'SS6004134']},
 'S5': {'normal': ['SS6004138'], 'tumor': ['SS6004139', 'SS6005041']},
 'S6': {'normal': ['SS6005042'], 'tumor': ['SS6005043', 'SS6005044']}}

In [5]:
normal_tumor = pd.DataFrame(dict_sample).T.reset_index()
for index, row in normal_tumor.iterrows():
    argument = ''
    name_file = f'{row["index"]}_'
    output = f'{row["index"]}_'
    for i in row['tumor']:
        argument+=f'-I SN_{i}.DR.bam '
        name_file+=f'{i}_'
        output+=f'{i}_'
    for j in row['normal']:
        argument+=f'-I SN_{j}_DR.bam '
        argument+=f'-normal {j}.DR.bam '
        name_file+=f'{j}.txt'
        output+=f'{j}_'
    
    output+=f'somatic.vcf.gz'
    argument+=f'-O {output}'
    print(argument)
    print(name_file)
    f = open(name_file, "a")
    f.write(argument)
    f.close()

-I SN_SS6004099.DR.bam -I SN_SS6004104.DR.bam -I SN_SS6004094_DR.bam -normal SS6004094.DR.bam -O S1_SS6004099_SS6004104_SS6004094_somatic.vcf.gz
S1_SS6004099_SS6004104_SS6004094.txt
-I SN_SS6004113.DR.bam -I SN_SS6004114.DR.bam -I SN_SS6004118.DR.bam -I SN_SS6004119.DR.bam -I SN_SS6004109_DR.bam -normal SS6004109.DR.bam -O S2_SS6004113_SS6004114_SS6004118_SS6004119_SS6004109_somatic.vcf.gz
S2_SS6004113_SS6004114_SS6004118_SS6004119_SS6004109.txt
-I SN_SS6004124.DR.bam -I SN_SS6004129.DR.bam -I SN_SS6004123_DR.bam -normal SS6004123.DR.bam -O S3_SS6004124_SS6004129_SS6004123_somatic.vcf.gz
S3_SS6004124_SS6004129_SS6004123.txt
-I SN_SS6004133.DR.bam -I SN_SS6004134.DR.bam -I SN_SS6004128_DR.bam -normal SS6004128.DR.bam -O S4_SS6004133_SS6004134_SS6004128_somatic.vcf.gz
S4_SS6004133_SS6004134_SS6004128.txt
-I SN_SS6004139.DR.bam -I SN_SS6005041.DR.bam -I SN_SS6004138_DR.bam -normal SS6004138.DR.bam -O S5_SS6004139_SS6005041_SS6004138_somatic.vcf.gz
S5_SS6004139_SS6005041_SS6004138.txt
-I S

In [6]:
normal_tumor = pd.DataFrame(dict_sample).T.reset_index()
for index, row in normal_tumor.iterrows():
    for j in row['normal']:        
        for i in row['tumor']:
            argument = ''
            name_file = f'{row["index"]}_'
            output = f'{row["index"]}_'
            argument+=f'-I SN_{i}.DR.bam '
            argument+=f'-I SN_{j}_DR.bam '
            argument+=f'-normal {j}.DR.bam '
            name_file+=f'{i}_'            
            name_file+=f'{j}.txt'
            output+=f'{i}_'
            output+=f'{j}_'
            output+=f'somatic.vcf.gz'
            argument+=f'-O {output}'
            print(argument)
            print(name_file)
            f = open(name_file, "a")
            f.write(argument)
            f.close()

-I SN_SS6004099.DR.bam -I SN_SS6004094_DR.bam -normal SS6004094.DR.bam -O S1_SS6004099_SS6004094_somatic.vcf.gz
S1_SS6004099_SS6004094.txt
-I SN_SS6004104.DR.bam -I SN_SS6004094_DR.bam -normal SS6004094.DR.bam -O S1_SS6004104_SS6004094_somatic.vcf.gz
S1_SS6004104_SS6004094.txt
-I SN_SS6004113.DR.bam -I SN_SS6004109_DR.bam -normal SS6004109.DR.bam -O S2_SS6004113_SS6004109_somatic.vcf.gz
S2_SS6004113_SS6004109.txt
-I SN_SS6004114.DR.bam -I SN_SS6004109_DR.bam -normal SS6004109.DR.bam -O S2_SS6004114_SS6004109_somatic.vcf.gz
S2_SS6004114_SS6004109.txt
-I SN_SS6004118.DR.bam -I SN_SS6004109_DR.bam -normal SS6004109.DR.bam -O S2_SS6004118_SS6004109_somatic.vcf.gz
S2_SS6004118_SS6004109.txt
-I SN_SS6004119.DR.bam -I SN_SS6004109_DR.bam -normal SS6004109.DR.bam -O S2_SS6004119_SS6004109_somatic.vcf.gz
S2_SS6004119_SS6004109.txt
-I SN_SS6004124.DR.bam -I SN_SS6004123_DR.bam -normal SS6004123.DR.bam -O S3_SS6004124_SS6004123_somatic.vcf.gz
S3_SS6004124_SS6004123.txt
-I SN_SS6004129.DR.bam -I S